
<h1 style="font-family:verdana;font-size:300%;text-align:center;background-color:#f2f2f2;color:#0d0d0d">MMI_2024_NLP - Week 1</h1>

<h1 style="font-family:verdana;font-size:180%;text-align:Center;color:#993333"> Lab 2: Introduction to wordvectors </h1>
<h3> Jeremie Mabiala </h3>


Before we start, please change the name of the notebook to the following format : **Firstname_LASTNAME_Lab2_intro_to_wordvectors.ipynb**


In some cells and files you will see code blocks that look like this:

```python
##############################################################################
#                    TODO: Write the equation for a line                     #
##############################################################################
pass
##############################################################################
#                              END OF YOUR CODE                              #
##############################################################################
```

You should replace the `pass` statement with your own code and leave the blocks intact, like this:

```python
##############################################################################
#                    TODO: Write the equation for a line                     #
##############################################################################
y = m * x + b
##############################################################################
#                              END OF YOUR CODE                              #
##############################################################################
```

In [36]:
import io, sys
import numpy as np

In [38]:
#  Activate the code below if you are using colab and your data files are in your drive.
# I worked locally.


# from google.colab import drive
# drive.mount("drive")


In [39]:
# ls drive


In [3]:

def load_vectors(filename):
    fin = io.open(filename, 'r', encoding='utf-8', newline='\n')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = np.asarray([float(x) for x in tokens[1:]])
    return data

In [40]:
# Loading word vectors

print('')
print(' ** Word vectors ** ')
print('')

'''
word_vectors is a dictionary that maps words to their numerical word vector
[word (string)] = [np-array]
'''

data_pth = "/Users/jeremie/Documents/AMMI-2024/AMMI2024/NLP/Lab2/"
word_vectors = load_vectors(data_pth + "wiki.en.vec")

tree_vector = word_vectors['tree']
print(type(tree_vector), len(tree_vector))


 ** Word vectors ** 

<class 'numpy.ndarray'> 300


In [41]:
## This function computes the cosine similarity between vectors u and v

def cosine(u, v):
    '''
    Parameters:
    u : 1-D numpy array
    v : 1-D numpy array

    Returns:
    cos (float) : value of the cosine similairy between vectors u, v
    '''
    ##########################################################################
    #                      TODO: Implement this function                     #
    ##########################################################################
    # Replace "pass" statement with your code
    cos = np.dot(u,v)/(np.linalg.norm(u)* np.linalg.norm(v))
    return cos
    ##########################################################################
    #                            END OF YOUR CODE                            #
    ##########################################################################

In [42]:
# compute similarity between words
print (f"test similarity {cosine(np.array([1,0,0]),np.array([1,0,0]))}", )
print('similarity(apple, apples) = %.3f' %
      cosine(word_vectors['apple'], word_vectors['apples']))
print('similarity(apple, banana) = %.3f' %
      cosine(word_vectors['apple'], word_vectors['banana']))
print('similarity(apple, tiger) = %.3f' %
      cosine(word_vectors['apple'], word_vectors['tiger']))

test similarity 1.0
similarity(apple, apples) = 0.637
similarity(apple, banana) = 0.431
similarity(apple, tiger) = 0.212


In [47]:
## Functions for nearest neighbor
## This function returns the word corresponding to
## nearest neighbor vector of x
## The list exclude_words can be used to exclude some
## words from the nearest neighbors search

def nearest_neighbor(x, word_vectors, exclude_words=[]):
    '''
    Parameters:
    x (string): word to find its nearest neighbour
    word_vectors (Python dict): {word (string): np-array of word vector}
    exclude_words (list of strings): words to be excluded from the search

    Returns:
    best_word (string) : the word whose word vector is the nearest neighbour
    to the word vector of x
    '''
    best_score = -1.0
    best_word = None
    ##########################################################################
    #                      TODO: Implement this function                     #
    ##########################################################################
    # Replace "pass" statement with your code
    for k in word_vectors.keys():
        score = cosine(x,word_vectors[k])

        if score> best_score and k not in exclude_words:
            best_score = score
            best_word = k
    return best_word
    ##########################################################################
    #                            END OF YOUR CODE                            #
    ##########################################################################

In [50]:
print('')
print('The nearest neighbor of cat is: ' +
      nearest_neighbor(word_vectors['cat'], word_vectors, exclude_words = ['cat', 'cats']))


The nearest neighbor of cat is: dog


#### Hint (using python priorty queues with the heapq datastructure):
if you don't want to store all the words and scores you can use the priortiy queue and only store the best K element so far.

In [55]:
## This function return the words corresponding to the
## K nearest neighbors of vector x.
## You can use the functions heappush and heappop.
import heapq
def knn(x, vectors, k):
    '''
    Parameters:
    x (string): word to find its nearest neighbour
    word_vectors (Python dict): {word (string): np-array of word vector}
    k (int): number of nearest neighbours to be found

    Returns:
    k_nearest_neighbors (list of tuples): [(score, word), (score, word), ....]
    '''

    k_nearest_neighbors = None
    ##########################################################################
    #                      TODO: Implement this function                     #
    ##########################################################################
    # Replace "pass" statement with your code
    heap = []
    exclude_words=[]
    for _ in range(k+1):
        best_word = nearest_neighbor(x, vectors,exclude_words)
        exclude_words.append(best_word)
        heap.append((cosine(x,word_vectors[best_word]),best_word))
        
    return heap[1:]

    ##########################################################################
    #                            END OF YOUR CODE                            #
    ##########################################################################

In [56]:
knn_cat = knn(word_vectors['cat'], word_vectors, 5)
print('')
print('cat')
print('--------------')
for score, word in knn(word_vectors['cat'], word_vectors, 5):
    print (word + '\t%.3f' % score)


cat
--------------
cats	0.732
dog	0.638
pet	0.573
rabbit	0.549
dogs	0.538


#### Hint:
To find the analogies, we find the nearest neighbour associated with the wordvector d
$$ d = \frac{c}{\Vert {c} \Vert} + \frac{b}{\Vert {b} \Vert} - \frac{a}{\Vert {a} \Vert}$$


In [58]:
## This function return the words d, such that a:b and c:d
## verifies the same relation

def analogy(a, b, c, word_vectors):
    '''
    Parameters:
    a (string): word a
    b (string): word b
    c (string): word c
    word_vectors (Python dict): {word (string): np-array of word vector}

    Returnrs:
    the word d (string) associated with c such that c:d is similar to a:b

    '''
    ##########################################################################
    #                      TODO: Implement this function                     #
    ##########################################################################
    # Replace "pass" statement with your code
    a = a.lower()
    b = b.lower()
    c = c.lower()
    
    result_words = None
    result_score = float('-Inf')
    
    x_a = word_vectors[a]/np.linalg.norm(word_vectors[a])
    x_b = word_vectors[b]/np.linalg.norm(word_vectors[b])
    x_c = word_vectors[c]/np.linalg.norm(word_vectors[c])

    for key in word_vectors:
        ## exclude some words
        if True in [i in key for i in [a,b,c]] :
            continue
        normalized = word_vectors[key]/np.linalg.norm(word_vectors[key])
        result = np.dot((x_c+x_b-x_a),normalized)

        if result > result_score:
            result_score = result
            result_words = key
    
    return result_words 
    ##########################################################################
    #                            END OF YOUR CODE                            #
    ##########################################################################

In [64]:
# Word analogies
print('')
print('france - paris + rome = ' + analogy('paris', 'france', 'rome', word_vectors))
print()
print('father - mother + niece = ' + analogy('mother', 'father', 'niece', word_vectors))


france - paris + rome = italy

father - mother + niece = nephew


## A word about biases in word vectors

In [65]:
## A word about biases in word vectors:
print('')
print('similarity(genius, man) = %.3f' %
      cosine(word_vectors['man'], word_vectors['genius']))
print('similarity(genius, woman) = %.3f' %
      cosine(word_vectors['woman'], word_vectors['genius']))


similarity(genius, man) = 0.445
similarity(genius, woman) = 0.325


In [72]:
## Compute the association strength between:
##   - a word w
##   - two sets of attributes A and B

def association_strength(w, A, B, vectors):
    '''
    Parameters:
    w (string): word w
    A (list of strings): The words belonging to set A
    B (list of strings): The words belonging to set B
    vectors (Python dict): {word (string): np-array of word vector}

    Returnrs:
    strength (float): the value of the association strength
    '''
    strength = 0.0
    part_a = 0.0
    part_b = 0.0
    ##########################################################################
    #                      TODO: Implement this function                     #
    ##########################################################################
    # Replace "pass" statement with your code
    strength_a, strength_b = .0, .0
    for a in A:
        part_a += ((1/len(A))*cosine(vectors[w],vectors[a]))
    for b in B:
        part_b += ((1/len(B))*cosine(vectors[w],vectors[b]))
        
    strength = part_a - part_b
    return strength 
    ##########################################################################
    #                            END OF YOUR CODE                            #
    ##########################################################################

In [73]:
## Perform the word embedding association test between:
##   - two sets of words X and Y
##   - two sets of attributes A and B

def weat(X, Y, A, B, vectors):
    '''
    Parameters:
    X (list of strings): The words belonging to set X
    Y (list of strings): The words belonging to set Y
    A (list of strings): The words belonging to set A
    B (list of strings): The words belonging to set B
    vectors (Python dict): {word (string): np-array of word vector}

    Returns:
    score (float): the value of the group association strength
    '''

    score = 0.0
    ##########################################################################
    #                      TODO: Implement this function                     #
    ##########################################################################
    # Replace "pass" statement with your code
    score_x = 0.0
    score_y = 0.0
    ## FILL CODE
    for i in X:
        score_x += association_strength(i,A,B,vectors)
    for i in Y:
        score_y += association_strength(i,A,B,vectors)
    score = score_x - score_y

    return score
    ##########################################################################
    #                            END OF YOUR CODE                            #
    ##########################################################################

In [74]:
career = ['executive', 'management', 'professional', 'corporation',
          'salary', 'office', 'business', 'career']
family = ['home', 'parents', 'children', 'family',
          'cousins', 'marriage', 'wedding', 'relatives']
male = ['john', 'paul', 'mike', 'kevin', 'steve', 'greg', 'jeff', 'bill']
female = ['amy', 'joan', 'lisa', 'sarah', 'diana', 'kate', 'ann', 'donna']

print('')
print('Word embedding association test: %.3f' %
      weat(career, family, male, female, word_vectors))


Word embedding association test: 0.847


## Word translation using word vectors

In the following, we will use word vectors in English and French to translate words from English to French. The idea is to learn a linear function that maps English word vectors to their correponding French word vectors. To learn this linear mapping, we will use a small bilingual lexicon, that contains pairs of words in English and French that are translations of each other.

The following function will load the small English-French bilingual lexicon:

In [75]:
def load_lexicon(filename):
    '''
    Parameters:
    filename(string): the path of the lexicon

    Returns:
    data(list of pairs of string): the bilingual lexicon
    '''
    fin = io.open(filename, 'r', encoding='utf-8', newline='\n')
    data = []
    for line in fin:
        a, b = line.rstrip().split(' ')
        data.append((a, b))
    return data

In [78]:
word_vectors_en = load_vectors(data_pth + 'wiki.en.vec')
word_vectors_fr = load_vectors(data_pth + 'wiki.fr.vec')
lexicon = load_lexicon(data_pth + "lexicon-en-fr.txt")
print(lexicon[:5])

[('the', 'le'), ('the', 'les'), ('the', 'la'), ('and', 'et'), ('was', 'fut')]


In [77]:
# We split the lexicon into a train and validation set
train = lexicon[:5000]
valid = lexicon[5000:5100]

The following function will learn the mapping from English to French. The idea is to build two matrices $X_{\text{en}}$ and $X_{\text{fr}}$, and to find a mapping $M$ that minimizes $||X_{\text{en}} W - X_{\text{fr}} ||_2$. In numpy, this mapping can be obtained by using the `numpy.linalg.lstsq` function.

In [170]:
def align(word_vectors_en, word_vectors_fr, lexicon):
    '''
    Parameters:
    word_vectors_en(dict: string -> np.array): English word vectors
    word_vectors_en(dict: string -> np.array): French word vectors
    lexicon(list of pairs of string): bilingual training lexicon

    Returns
    mapping(np.array): the mapping from English to French vectors
    '''
    x_en, x_fr = [], []
    ##########################################################################
    #                      TODO: Implement this function                     #
    ##########################################################################
    # Replace "pass" statement with your code

    # Create mapping arrayas: X_en and X_fr
    for en_word, fr_word in lexicon:
        if en_word in word_vectors_en or fr_word in word_vectors_fr:
            x_en.append(word_vectors_en[en_word])
            x_fr.append(word_vectors_fr[fr_word])

    X_en = np.array(x_en)
    X_fr = np.array(x_fr)

    W, residuals, _, _ = np.linalg.lstsq( X_en , X_fr, rcond=None)


    return W  , residuals.reshape(-1)


    ##########################################################################
    #                            END OF YOUR CODE                            #
    ##########################################################################

In [171]:
mapping = align(word_vectors_en, word_vectors_fr, lexicon)
mapping

(array([[-0.06183285, -0.01071552,  0.00175985, ..., -0.01107046,
          0.01629405, -0.01644996],
        [-0.01655313, -0.02930488,  0.09810107, ..., -0.01744702,
         -0.02848298,  0.02070179],
        [-0.01970861, -0.0147154 ,  0.01231819, ...,  0.03036093,
         -0.00209909, -0.00944313],
        ...,
        [ 0.0669847 ,  0.02351181,  0.02041902, ...,  0.00886501,
          0.08635366,  0.00595836],
        [ 0.01936122,  0.00552446,  0.01234669, ..., -0.00623332,
         -0.05116348,  0.05634361],
        [ 0.00530333, -0.03424679, -0.03369923, ..., -0.01344391,
         -0.00051053, -0.00491391]]),
 array([240.83364785, 229.76001971, 240.87553006, 242.85647259,
        243.71812188, 226.23758728, 253.75036774, 232.27915867,
        240.37369797, 242.5454353 , 234.87123906, 242.73885799,
        240.05196358, 225.23258537, 231.97861248, 275.50632002,
        254.2120771 , 212.69817596, 231.11341292, 236.77024946,
        228.47970302, 230.40315553, 231.74205474, 254

Given a mapping, a set of word English word vector and French word vectors, the next function will translate the English word to French. To do so, we apply the mapping on the English word, and retrieve the nearest neighbor of the obtained vector in the set of French word vectors. The translation is then the corresponding French word.

In [172]:
def translate(word, word_vectors_en, word_vectors_fr, mapping):
    '''
    Parameters:
    word(string): an English word
    word_vectors_en(dict: string -> np.array): English word vectors
    word_vectors_en(dict: string -> np.array): French word vectors
    mapping(np.array): the mapping from English to French vectors

    Returns
    A string containing the translation of the English word
    '''
    ##########################################################################
    #                      TODO: Implement this function                     #
    ##########################################################################
    # Replace "pass" statement with your code
    if word in word_vectors_en:
        en_vector = word_vectors_en[word]
        fr_vector = np.dot(en_vector, mapping[0])

        # Find the closest French word in word_vectors_fr
        min_distance = float('inf')
        translation = None

        for fr_word, fr_word_vector in word_vectors_fr.items():
            distance = np.linalg.norm(fr_vector - fr_word_vector)
            if distance < min_distance:
                min_distance = distance
                translation = fr_word

        return translation
    
    else:
        return None
    
    ##########################################################################
    #                            END OF YOUR CODE                            #
    ##########################################################################

In [174]:
print(translate("man", word_vectors_en, word_vectors_fr, mapping))
print(translate("lady", word_vectors_en, word_vectors_fr, mapping))
print(translate("learning", word_vectors_en, word_vectors_fr, mapping))

,
maîtresse
apprendre


Finally, let's implement a function to evaluate this method on the validation lexicon:

In [178]:
def evaluate(valid_data, word_vectors_en, word_vectors_fr, mapping):
    '''
    Parameters:
    valid(a list of pairs of string): the validation lexicon
    word_vectors_en(dict: string -> np.array): English word vectors
    word_vectors_en(dict: string -> np.array): French word vectors
    mapping(np.array): the mapping from English to French vectors

    Returns
    Accuracy(float): the accuracy on the validation lexicon
    '''
    acc, n = 0.0, 0
    ##########################################################################
    #                      TODO: Implement this function                     #
    ##########################################################################
    # Replace "pass" statement with your code
    for en_word, fr_word in valid_data:
        # Translate the English word to French using thhe mapping above:
        translated = translate(en_word, word_vectors_en, word_vectors_fr, mapping)
        
        # Check if the translated  matches the expected French word
        if translated == fr_word:
            acc += 1
        n += 1
    
    # Compute accuracy
    acc = acc / n if n > 0 else 0.0
    return acc
    ##########################################################################
    #                            END OF YOUR CODE                            #
    ##########################################################################

In [179]:
evaluate(valid, word_vectors_en, word_vectors_fr, mapping)

0.54

In [180]:
'homme' in word_vectors_fr

True